In [ ]:
import together
from together import Together
TOGETHER_API_KEY='73989ea4aa7a495b1fed7c7edec40398e6ce1525f7049afd85060a0e041058e2'
path='/Users/salvador/GitHub/absa-travelogues/ABSA_data_ok/'
link_modelo = Together(api_key=TOGETHER_API_KEY)
modelo="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"

In [ ]:
 contenido_sys = "You are ahistorian and literary scholar with expertise on historical travel literature and colonial literature and labelling named entities"
question = "Extract the relevant named entities from the given sentence."
chain_of_thought = """
Task: Extract relevant named entities from the given sentence based on the following labels:
- person: proper names of people
- location: proper names of locations
- fauna: common and scientific names of animals and fauna
- flora: common and scientific names of plants and flora
- weather_phenomena: descriptions of weather phenomena and climate
- organisation: proper names of organizations
- biomes: names of biomes
- mythological_entities: fantastical entities, deities, or God
- human_landform: landforms influenced by people
- natural_landform: landforms not influenced by people
- natural_phenomenon: dynamic and naturally occurring phenomena
- land_cover: elements that cover the land

Only respond in JSON format, Do not add any more comment.
The structure of the JSON format is like this:
{
        "person": ["entity_1", "entity_2", "entity_n"],
        "location": ["entity_1", "entity_2", "entity_n"],
        "fauna": ["entity_1", "entity_2", "entity_n"],
        "flora": ["entity_1", "entity_2", "entity_n"],
        "weather_phenomena": ["entity_1", "entity_2", "entity_n"],
        "organisation": ["entity_1", "entity_2", "entity_n"],
        "biomes": ["entity_1", "entity_2", "entity_n"],
        "mythological_entities": ["entity_1", "entity_2", "entity_n"],
        "human_landform": ["entity_1", "entity_2", "entity_n"],
        "natural_landform": ["entity_1", "entity_2", "entity_n"],
        "natural_phenomenon": ["entity_1", "entity_2", "entity_n"],
        "land_cover": ["entity_1", "entity_2", "entity_n"],
        }
Let's approach this step-by-step:

Example 1:
Sentence: <<<The Amazon rainforest is home to jaguars and giant water lilies, while the Andes Mountains tower above.>>>

Step 1: Identify potential named entities
- Amazon rainforest
- jaguars
- giant water lilies
- Andes Mountains

Step 2: Categorize each entity and asign a label
- Amazon rainforest: biomes (rainforest is a type of biome)
- jaguars: fauna (common name of an animal)
- giant water lilies: flora (common name of a plant)
- Andes Mountains: natural_landform (mountain range not influenced by people)

Step 3: Format the output

   {"person": [],
    "location": [],
    "fauna": ["jaguars"],
    "flora": ["giant water lilies"],
    "weather_phenomena": [],
    "organisation": [],
    "biomes": ["Amazon rainforest"],
    "mythological_entities": [],
    "human_landform": [],
    "natural_landform": ["Andes Mountains"],
    "natural_phenomenon": [],
    "land_cover": [],
   }


Example 2:
Sentence: <<<Hurricane Katrina devastated New Orleans in 2005, causing widespread flooding and destruction.>>>

Step 1: Identify potential named entities
- Hurricane Katrina
- New Orleans

Step 2: Categorize each entity and asign a label
- Hurricane's Katrina: weather_phenomena (description of a weather event)
- New Orleans: location (proper name of a city)

Step 3: Format the output

   {"person": [],
    "location": ["New Orleans"],
    "fauna": [],
    "flora": [],
    "weather_phenomena": ["Hurricane\\'s' Katrina"],
    "organisation": [],
    "biomes": [],
    "mythological_entities": [],
    "human_landform": [],
    "natural_landform": [],
    "natural_phenomenon": [],
    "land_cover": [],
  }

Example 3:
Sentence: <<<The Great Wall of China stretches across the Gobi Desert, where Bactrian camels roam freely.>>>

Step 1: Identify potential named entities
- Great Wall of China
- Gobi Desert
- Bactrian camels

Step 2: Categorize each entity and asign a label
- Great Wall of China: human_landform (man-made structure influencing the landscape)
- Gobi Desert: biomes (desert is a type of biome)
- Bactrian camels: fauna (common name of an animal)

Step 3: Format the output

   {"person": [],
    "location": [],
    "fauna": ["Bactrian camels"],
    "flora": [],
    "weather_phenomena": [],
    "organisation": [],
    "biomes": ["Gobi Desert"],
    "mythological_entities": [],
    "human_landform": ["Great Wall of China"],
    "natural_landform": [],
    "natural_phenomenon": [],
    "land_cover": [],
  }


Now, DO NOT take into account the previous examples, use them JUST AS A REFERENCE. 

The sentence for analyzing is:


"""

In [ ]:

## Generation

def llm_inference(modelo,mensaje):
  TOGETHER_API_KEY='73989ea4aa7a495b1fed7c7edec40398e6ce1525f7049afd85060a0e041058e2'
  link_modelo = Together(api_key=TOGETHER_API_KEY)
  output =[]
  extract = link_modelo.chat.completions.create(
      messages=mensaje,
      model=modelo,
      #top_p=1,
      #top_k=40,
      temperature= 0.0,
      #response_format={
      #   "type": "json_object",
      #   "schema": characters.model_json_schema(),},
  )
  try:
    output = json.loads(extract.choices[0].message.content)
  except ValueError as e:
    print ('Error en JSON tipo : ',e)
    print(output)
    extract = link_modelo.chat.completions.create(
              messages=[
                  {
                      "role": "system",
                      "content": ' You are a helpfull assistant expert on JSON format.Your work is to correct this JSON to be a correct JSON text.Only repond in a JSON format. Do not add any more comment.',
                  },
                  {
                      "role": "user",
                      "content":f'This is the uncorrect JSON text:  {extract.choices[0].message.content}',
                  },
              ],
              model=modelo,
              temperature= 0.0,
            )
    try:
          output = json.loads(extract.choices[0].message.content)
    except ValueError as e:
        print (f'Segundo  error, skipe Sentence. Tipo de error: ',e)
        output=[]

  #print(extract.choices[0].message.content)
  #print(json.dumps(output, indent=1))
  #print(output)
  return(output)